In [105]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow warnings https://stackoverflow.com/a/40871012
from deepface import DeepFace
import subprocess
import numpy as np
from decimal import Decimal # for proper rounding
import random
import time
import pandas as pd
from datetime import datetime
import struct


# CONSTANTS
EXECUTABLE_PATH = "ABY/build/bin"
INPUT_FILE_NAME = "input_vecs.txt"
OUTPUT_FILE_NAME = "output.txt"
EXECUTABLE_NAME = 'cos_sim_simd_test'
EXECUTABLE_NAME_SCENARIO = 'cos_dist'
CMD = f"./{EXECUTABLE_NAME} -r 0 -f {INPUT_FILE_NAME} -o {OUTPUT_FILE_NAME} & (./{EXECUTABLE_NAME} -r 1 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"
CMD_SCENARIO = f"./{EXECUTABLE_NAME_SCENARIO} -r 1 -f {INPUT_FILE_NAME} -o {OUTPUT_FILE_NAME} & (./{EXECUTABLE_NAME_SCENARIO} -r 0 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"
ITERATIONS = 1

In [115]:
def get_cos_dist_aby(x, y):
    # write two vectors to a file
    with open(f"{EXECUTABLE_PATH}/{INPUT_FILE_NAME}", 'w') as f:
        for x_i, y_i in zip(x, y):
            f.write(f"{x_i} {y_i}\n")
    
    # execute the ABY cos sim computation
    start = time.time()
    output = subprocess.run(CMD, shell=True, capture_output=True, text=True, cwd=EXECUTABLE_PATH)
    end = time.time()
    assert (output.returncode == 0) # make sure the process executed successfully
    
    # read the result from the output file
    with open(f"{EXECUTABLE_PATH}/{OUTPUT_FILE_NAME}") as f:
        return float(round(Decimal(str(f.read())),6)), end-start
    
def get_cos_dist_aby_scenario(x, y, r):
    # write two vectors to a file
    with open(f"{EXECUTABLE_PATH}/{INPUT_FILE_NAME}", 'w') as f:
        for x_i, y_i in zip(x, y):
            f.write(f"{x_i} {y_i}\n")
    y_1 = r # random nonces
    y_0 = np.bitwise_xor(y.astype(int), y_1.astype(int)) # md's share
    print(y_0)
    print(y_1)
    with open(f"{EXECUTABLE_PATH}/share0.txt", 'w') as f:
        for i in y_0:
            f.write(f"{i}\n")
    with open(f"{EXECUTABLE_PATH}/share1.txt", 'w') as f:
        for i in y_1:
            f.write(f"{i}\n")
            
    # execute the ABY cos sim computation
    start = time.time()
    output = subprocess.run(CMD_SCENARIO, shell=True, capture_output=True, text=True, cwd=EXECUTABLE_PATH)
    end = time.time()
    assert (output.returncode == 0) # make sure the process executed successfully
    
    # read the result from the output file
    with open(f"{EXECUTABLE_PATH}/{OUTPUT_FILE_NAME}") as f:
        return f.read(), end-start, output
        
    
def get_cos_dist_numpy(x, y):
    start = time.time()
    result = 1 - np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))
    end = time.time()
    # print(result)
    # do some magic of casting to str then Decimal then float to round properly
    result = float(round(Decimal(str(result)),6))
    return result, end-start

def get_cos_dist_deepface(img1, img2):
    result = DeepFace.verify(img1_path = img1, img2_path = img2, model_name="SFace", enforce_detection=True, distance_metric="cosine")["distance"]
    return float(round(Decimal(str(result)),6))

def round_to_6(x):
    return float(round(Decimal(str(x)),6))

def get_embedding(imagepath):
    return DeepFace.represent(img_path = imagepath, model_name="SFace", enforce_detection=True)[0]["embedding"]

def get_two_random_images_and_their_embeddings(same_person):
    people = os.listdir('lfw') # list of all people that have images
    people_with_multiple_images = [p for p in people if len(os.listdir(f"lfw/{p}")) > 1] # list of people with more than one image in folder
    embedding1, embedding2 = None, None # face embeddings
    while embedding1 is None or embedding2 is None: # try until the chosen images have detectable faces
        try:
            if same_person:
                # same person should have more than one image (we might still end up choosing the same image of that person with prob 1/n, but that's ok)
                person1 = random.choice(people_with_multiple_images)
                person2 = person1
            else:
                # two persons chosen should be different
                person1 = random.choice(people)
                person2 = random.choice([p for p in people if p != person1])
            # get two random images
            img1 = f"lfw/{person1}/{random.choice(os.listdir(f'lfw/{person1}'))}"
            img2 = f"lfw/{person2}/{random.choice(os.listdir(f'lfw/{person2}'))}"
            # try to extract embeddings from both images
            embedding1 = get_embedding(img1)
            embedding2 = get_embedding(img2)
        except Exception as e:
            # failed to detect faces in images, try again
            # print(e)
            pass
    return img1, embedding1, img2, embedding2

def verify_computation(same_person):
    # get two random images
    img1, img2 = get_two_random_images(same_person)
    print(f"{img1=}, {img2=}")
    # extract embeddings, disregard pictures where the face cannot be detected
    try:
        x = get_embedding(f"lfw/{img1}")
        y = get_embedding(f"lfw/{img2}")
    except ValueError as e:
        print(e)
        return 0
    except Exception as e:
        # print(e)
        return 0
    # print(f"{x=}, {y=}")
    # compute the cosine similarity
    cs = cos_sim(x,y)
    cs_aby = cos_sim_aby(x,y)
    cs_deepface = cos_dist_deepface(img1, img2)
    print(f"{cs=}, {cs_aby=}, {cs_deepface=}")
    print("----------")
    # make sure the results are the same
    assert (cs == cs_aby == cs_deepface)

def run_verification(same_person):
    # get two images and their embeddings to compare
    img1, emb1, img2, emb2 = get_two_random_images_and_their_embeddings(same_person)
    # compute the cos dist with numpy and ABY (with timings)
    cos_dist_numpy, time_numpy = get_cos_dist_numpy(emb1, emb2)
    cos_dist_aby, time_aby = get_cos_dist_aby(emb1, emb2)
    result = DeepFace.verify(img1_path = img1, img2_path = img2, model_name="SFace", enforce_detection=True, distance_metric="cosine")
    cos_dist_df = round_to_6(result["distance"])
    threshold = result["threshold"]
    # verify the results are the same (sanity check against the deepface module in case both numpy and aby are wrong)
    # assert (cos_dist_numpy == cos_dist_aby == cos_dist_df), f"{cos_dist_numpy} == {cos_dist_aby} == {cos_dist_df}"
    aby_ver = cos_dist_aby < threshold
    # put the results in a dataframe according to the scheme in the notebook
    return [same_person,result["verified"],aby_ver,cos_dist_numpy,cos_dist_aby,threshold,img1,img2,time_numpy,time_aby]

def fxor(a, b):
    rtrn = []
    a = struct.pack('d', a)
    b = struct.pack('d', b)
    for ba, bb in zip(a, b):
        rtrn.append(ba ^ bb)
    return struct.unpack('d', bytes(rtrn))[0]

def fxor_list(a, b):
    assert len(a) == len(b)
    rtrn = []
    for i in range(len(a)):
        rtrn.append(fxor(a[i], b[i]))
    return rtrn

def scale(x, factor=100, decimals=2):
    """Scale the float(s) by `factor` and round to 'decimals' decimal places """
    return x.round(decimals=decimals)*factor


In [63]:
r = np.random.uniform(low=-3.0, high=3.0, size=(128,))
s = np.random.uniform(low=-3.0, high=3.0, size=(128,))
r = scale(r)
s = scale(s)
r

array([   7.,   84., -231., -198.,   -6.,   95.,  -25., -110., -162.,
       -246., -206., -256.,  252., -132., -202.,  -13.,  -97., -154.,
        -76.,   99.,    6.,  209.,  272., -264.,   32.,  153.,  158.,
       -119., -187.,  -43.,  -25., -132., -159.,  106.,  104.,   21.,
         77.,  278., -262.,   68.,  101.,   35.,  -30.,  153., -231.,
        153., -109.,  220., -186.,  246., -291., -293.,  111.,  -57.,
        241., -249.,  281.,  300., -134.,  177., -209.,  245., -193.,
       -180.,  -29.,    7., -141., -201.,  -38.,  -18.,   48., -156.,
        163.,  267.,  169., -232.,   80.,  274.,   44., -261.,   83.,
         45.,  -86., -295.,  142.,  176.,    3.,  113.,  -78.,   67.,
        229.,  162., -246.,  -66., -268., -251.,    9., -142., -110.,
        144.,   34.,  166.,  136., -280., -251., -207., -237.,  -60.,
        -43.,  217., -256., -104., -101., -115.,  157., -193.,  253.,
         76.,  -88.,   26.,   84.,  245., -210.,   59.,   73.,  -69.,
        -14.,  198.]

In [3]:
img1, x, img2, y = get_two_random_images_and_their_embeddings(False)
get_cos_dist_aby(x, y)

(0.571587, 4.88341498374939)

In [4]:
df = pd.DataFrame(columns=['real_ver','df_ver','aby_ver','df_dist','aby_dist','threshold','img1','img2', 'np_time', 'aby_time'])
dt_string = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
for _ in range(ITERATIONS):
    # append results of a run with the same person
    df.loc[len(df)] = run_verification(True)
    # run with different people
    df.loc[len(df)] = run_verification(False)
    df.to_csv(f"df_{dt_string}_{ITERATIONS=}")
    print(_)


0


In [5]:
df

,real_ver,df_ver,aby_ver,df_dist,aby_dist,threshold,img1,img2,np_time,aby_time
0,True,True,True,0.391714,0.391714,0.593,lfw/Richard_Armitage/Richard_Armitage_0001.jpg,lfw/Richard_Armitage/Richard_Armitage_0002.jpg,0.000593,4.956547
1,False,False,False,0.836785,0.836785,0.593,lfw/Giuseppe_Morchio/Giuseppe_Morchio_0001.jpg,lfw/Tom_Moss/Tom_Moss_0001.jpg,0.000372,5.265475


In [6]:
# Simulation of the protocol in plaintext


# y is the base image that should be distributed between Drone and MB, x is the current face captured

img1, x, img2, y = get_two_random_images_and_their_embeddings(False)

# split y into two shares

# create a vector of random nonces
r = [random.uniform(-2.0, 2.0) for _ in range(128)]
# y1 is the nonces
y1 = r.copy()
# y0 is y XOR random nonces
y0 = fxor_list(y, r)


In [7]:
def cos_dist_np_temp(x, y):
    result = 1 - np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return result

In [8]:
print(get_cos_dist_numpy(x, y))

(0.788146, 0.0002624988555908203)


In [ ]:
cos_dist_x_y0 = 1 - np.dot(x, y0)/(np.linalg.norm(x)*np.linalg.norm(y0))
cos_dist_x_y0

In [ ]:
r

In [ ]:
xor = lambda x,y:(x.view("int64")^y.view("int64")).view("float64")
print(xor(np.array(r),np.array(y)))

In [ ]:
# Simulation of the protocol in plaintext on ints

# y is the base image that should be distributed between Drone and MB, f is the current face captured

# Generate two random fake int embeddings
f = np.random.randint(-100,100,128)
y = np.random.randint(-100,100,128)

# split y into share

# y1 is the nonces
y1 = np.random.randint(-100,100,128)
# y0 is y XOR y1
y0 = np.bitwise_xor(y, y1)

# sanity check
y == np.bitwise_xor(y0, y1)

In [ ]:
# Drone computes distance between f and y1
drone_dist, t = get_cos_dist_numpy(f, y1)
# MD computes distance between f and y0
md_dist, t = get_cos_dist_numpy(f, y0)
drone_dist, md_dist, fxor(drone_dist, md_dist), get_cos_dist_numpy(f, y)

In [ ]:
def fxor(a, b):
    """
    Take two floats and xor their individual bits. Return the created float.
    """
    rtrn = []
    a = struct.pack('d', a)
    b = struct.pack('d', b)
    for ba, bb in zip(a, b):
        rtrn.append(ba ^ bb)
    return struct.unpack('d', bytes(rtrn))[0]

# another way of xoring individual bit of floats using numpy
fxor_np = lambda x,y:(x.view("int64")^y.view("int64")).view("float64")

# just so I get it, a very simplified scenario

x = 4 # captured face
y = 5 # base image
r = 6 # 'random' nonce
y1 = r # one share is the nonce
y0 = y ^ r # second share is y xor r
# take an operation (multiplying) that results in ints as well
a = y1*x
b = y0*x
print(f"This works, {a=}, {b=}, x*y == {x*y}, a^b == {a^b}")
# if intermediate results are floats (say we wanted to compute y/x), then xoring does not make sense
a = y1/x
b = y0/x
print(f"Now {a=} and {b=}, and y/x = {y/x}")
print(f"xoring floats does not make sense: {fxor(a,b)}")
# a^b gives " unsupported operand type(s) for ^: 'float' and 'float' "


In [ ]:
# this makes sense beacuse we are using ints
a = 4
b = 5
c = a^b
print(f"{c=}, c^a={c^a}, c^b={c^b}")
print((c+5)^a)

In [ ]:
# this does not makes sense, c becomes garbage and thus operations on it do not make sense
a = 0.4
b = 0.5
c = fxor(a, b)
# I can still xor it back to get the original value
print(f"{c=}, f3^a ={fxor(c,a)}, f3^b = {fxor(c,b)}")
print(fxor(c+2,a))

In [67]:
# generating shares

img1, x, img2, y = get_two_random_images_and_their_embeddings(False)
r = np.random.uniform(low=-3.0, high=3.0, size=(128,))
x = np.array(x)
print(x)
y = np.array(y)
print(get_cos_dist_numpy(x, y))
x = scale(x)
print(x)
y = scale(y)
r = scale(r)
result, exec_time, output = get_cos_dist_aby_scenario(x, y, r)

[-1.35394609  0.47900155  0.745942    0.8028484   0.9328922  -0.02735639
  0.01925261  0.72965533  0.48573205 -0.71765894  1.59478164  0.33729205
  0.46533918  0.28250113  0.31578583 -1.22140074 -0.5246681  -0.38301435
 -1.49788773  0.00915366 -0.85558581  0.68217993  0.40168104  0.17702484
  0.47998279  0.74102664 -1.94327664  0.80628729 -0.76856309  0.35238624
  0.96718884 -0.0732961   0.96404713 -0.5782274   0.14296725 -0.55136114
  0.29328087 -0.21023434 -1.01893151 -1.46633971  0.14188378 -0.4310171
  0.70894927  1.05156434  0.88598341  0.30429968 -0.10860778  0.91761231
  1.22610617 -1.00513053 -0.54970151  0.5759275   0.55477983 -0.15740024
 -0.51349515  0.658562   -0.08719099  0.91350776 -0.2945638  -0.74646318
  0.41274828  0.59300435 -0.957223   -0.11604135 -0.71197796  0.91579926
 -0.08310894 -1.13296318  0.07637653  0.43749776  0.56885558  0.3532486
  0.0279591  -0.17992286 -0.08559912 -0.13748053 -0.45621994  0.16312203
 -1.09760082 -0.54039228 -0.01557674 -0.43834591  0.0

In [122]:
rng = np.random.default_rng()
x = rng.integers(0, 2, 128)
y = rng.integers(0, 2, 128)
r = rng.integers(0, 2, 128)
get_cos_dist_numpy(x,y)

(0.495638, 0.0001583099365234375)

In [123]:
result, exec_time, output = get_cos_dist_aby_scenario(x, y, r)

[0 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0
 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1
 1 1 0 1 1 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 1 0 0
 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
[1 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1
 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1
 0 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1 0 1 1 1 1
 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1 0]


In [124]:
result

'0.495638\n'

In [125]:
print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: output.txt
s_product nvals: 128
s_product bitlen: 64
Online time is distributed as follows: 
Bool: local gates: 256.2000000000, interactive gates: 291.8070000000, layer finish: 189.0040000000
Yao: local gates: 5.7320000000, interactive gates: 4.7220000000, layer finish: 2.1860000000
Yao Rev: local gates: 5.0690000000, interactive gates: 4.3140000000, layer finish: 2.5030000000
Arith: local gates: 5.9130000000, interactive gates: 4.7280000000, layer finish: 5.4620000000
SPLUT: local gates: 5.6250000000, interactive gates: 4.6840000000, layer finish: 27.4190000000
Communication: 1569.3850000000

Complexities: 
Boolean Sharing: ANDs: 5975939 (1-bit) ; Depth: 10990
Total Vec AND: 5975939
Total Non-Vec AND: 5975939
XOR vals: 5896327 gates: 1386811
Comb gates: 0, CombStruct gates: 0, Perm gates: 0, Subset gates: 24576, Split gates: 0
Yao: ANDs: 0 ; Depth: 0
Reverse Yao: ANDs: 0 ; Depth: 0
Arithmetic Sharing: MULs: 0 ; Depth: 0
SP-LUT Sharing:

18.627936010197157